# Attention 뽀개기

https://pozalabs.github.io/transformer/ 를 읽어보며 정리한 jupyter notebook 입니다.

이장에선 transformer 논문을 요약하면서 추가적인 기법을 설명하도록 합니다.

## Why?

### Long-term dependency problem
- sequence data를 처리하기 위해 이전까지 많이 쓰이던 model 은 recurrent model 이였습니다. recurrent model 은 t번째에 대한 output 을 만들기 위해 t번째 input과 t-1 번쨰 hidden state 값을 이용했습니다. 이렇게 한다면 자연스럽게 문자으이 순차적인 특성이 유지가 되었습니다. 문장을 쓸 떄 뒤의 단어부터 쓰지 않고 처음부터 차례차례 쓰는 것과 마찬가지 인것입니다.
- 하지만 recurrent model 의 경우 많은 개선점이 있었음에도 long-term-dependency에 취약하다는 단점이 있었습니다. 예를 들어 "저는 언어학을 좋아하고, 인공지능중에서도 딥러닝을 배우고 있고 자연어 처리에 관심이 많습니다" 라는 문장을 만드는 것이 model의 task라고 합시다 이때 '자연어'단어를 만드는데 '언어학'이라는 단어는 중요한 단서입니다.
- 그러나 두 단어 사이의 거리가 가깝지 않으므로 model은 앞의 '언어학'이 라는 단어를이용해 '자연어'라는 단어를 만들지 못하고 , 언어학 보다 가까운 단어인 '딥러닝'을 보고 '이미지'를 만들 수도 있는것 입니다. 이처럼, 어떤정보와 다른 정보 사이의 거리가 멀 때 해당 정보를 이용하지 못하는것이 long-term dependecy problem 입니다.
- recurrent model은 순차적인 특성이 유지되는 뛰어난 장점이 있었음에도, long-term dependency problem 이라는 단점을 가지고 있습니다.
- 이와 달리 transformer는 recurrence 를 사용하지 않고 대신 attention mechanism만을 사용해 input 과 output의 dependency를 포착해냈습니다.

### Parallelization 
- recurrent model 은 학습 시 , t번째 hidden state를 얻기 위해서 t-1번째 hidden state가 필요 했습니다. 즉, 순서대로 계산될 필요가 있었습니다. 그래서 병렬 처리를 할 수 없었고 계산 속도가 느렸습니다.
( hidden state를 이용해 순서대로 계산됬기 때문에 속도가 느렸다 )
- 하지만 transformer에서는 학습 시 encoder에서는 각각의 position에 대해, 즉 각각의 단어에 대해 attention을 해주기만 하고, decoder에서는 masking 기법을 이용해 병렬 처리가 가능해집니다. (masking에 대해선 아래서 알려드림)


## Model Architecture

### Encoder and Decoder structure
![](https://pozalabs.github.io/assets/images/encoder-decoder.png)

- encoder는 input sequence (x1,...xn)에 대해 서로 다른 representation 인 z = (z1,....zn)으로 바꿔줍니다.
- decoder 는 z를 받아, output sequence(y1,...yn)를 하나씩 만들어 냅니다.
- 각각의 step에서 다음 symbol을 만들 때 이전에 만들어진 output(symbol)을 이용합니다. 예를 들어 "저는 사람입니다"라는 문장에서 사람입니다를 만들 때 '저는'이라는 symbol을 이용하는 겁니다.이런 특성을 auto-regressive하다고 합니다.


### Encoder and Decoder stacks 
![Encoder and Decoder stacks](https://pozalabs.github.io/assets/images/archi2.png)

### Encoder 
- N개의 동일한 Layer 로 구성돼 있습니다. input(x) 가 첫번째 layer에 들어가게 되고, layer(x)는 다시 layer에 들어가는 식입니다.
- 그리고 각각의 layer는 두개의 sub-layer, multi-head self-attention mechanism과 position-wise fully connected feed-foward network를 가지고 있습니다.
- 이때 두 개의 sub-layer에 residual connection을 이요합니다.  residual connection은 input을 output으로 그대로 전달하는 것을 말합니다. 이때 sub-layer의 output dimension을 embedding dimention과 맞춰줍니다. x+Sublayer(x)를 하기 위해서, 즉 residual connection 을 하기 위해서는 두 값의 차원을 맞춰야 할 필요가 있습니다. 그 후에  layer normalization을 적용합니다.

### Decoder 
- 역시 N개의 동일한 layer로 이루어져 있습니다.
- encoder와 달리 encoder의 결과에 multi-head attention 을 수행할 sub-layer를 추가합니다. 
- 마찬가지로 sub-layer에 residual connection을 사용한뒤, layer normalization 을 해줍니다.
- decoder에서는 encoder 와 달리 순차적으로 결과를 만들어내야 하기 때문에, self-attention을 변형합니다. 바로 masking을 해주는것입니다.  masking을 통해 position i 보다 이후에 있는 posititon에 attention을 주지 못하게 합니다. 즉, position i에 대한 예측은 미리 알고 있는 output들에만 의존을 하는 것입니다.  
![decoder_masking](https://pozalabs.github.io/assets/images/masking.png)

- 위의 예시를 보면, a를 예측할 때는 a이후에 있는 b,c에는 attention이 주어지지 않는 것 입니다. 그리고 b를 예측할 떄는 b이전에 있는 a만 attention이 주어질 수 있고 이후에 있는 c는 attention이 주어지지 않습니다.

### Embedding and Softmax
- embedding 값을 고정시키지 않고, 학습을 하면서 embedding값이 변경되는 learned embedding을 사용했습니다. 이때 input과 output은 같은 embedding layer를 사용합니다.
- 또한 decoder output을 다음 token의 확률로 바꾸기 위해 learned linear transformer과 softmax function을 사용했습니다. learned linear transformation을 사용했다는 것은 decoder output에 weight matrix W를 곱해주는데, 이때 W가 학습된다는 것입니다. 

### Attention
- attention은 단어의 의미처럼 특정 정보에 좀 더 주의를 기울이는 것입니다.
- 예를 들어 model이 수행해야 하는 task가 번역이라고 해봅시다. source는 영어이고, target은 한국어 입니다. "Hi my name is HSW" 라는 문장과 대응되는 "안녕, 내이름은 승우야"라는 문장이 있습니다. model이 (이름은)이라는 token을 decode할때 source에서 가장 중요한 것은 name 입니다.
- 그렇다면, source의 모든 token이 비슷한 중요도를 갖기 보다는 name이 더 큰 중요도를 가지면 되겠습니다. 이때, 더 큰 중요도를 갖게 만드는 방법이 바로 attention입니다. 

### Scaled Dot-Product Attention
![Scaled Dot-Product Attention](https://pozalabs.github.io/assets/images/sdpa.PNG)
- 논문에선 attention을 Scaled Dot-Product-Attention이라고 부릅니다. 수식을 살펴보면 이렇게 부르는 이유가 있습니다. 
![](img/20191020_234348.png)

- 먼저 input은 dk dimention의 query와 key들, dv dimention의 value들로 이루어져 있습니다.
- 이때 모든 query와 key에 대한 dot-product를 계산하고 각각을 sqrt(dk)로 나누어 줍니다. dot-product를 하고 sqrt(dk)로 scaling 을 해주기 때문에 Scaled Dot-Product Attention 이라고 합니다. 그리고 여기에 softmax를 적용해 value들에 weights를 얻어냅니다.
- Key와 Value는 Attention이 이루어지는 위치에 상관없이 같은 값을 갖게 됩니다.  이때 query 와 key에 대한 dot-product를 계산하면 각각의 query와 key 사이의 유사도를 구할 수 있게 됩니다. 흔히 들어본 cosine similarity는 dot-product에서 vector의 magnitude로 나눈 것 입니다. (sqrt(dk)로 scaling 을 해주는 이유는 dot-products의 값이 커질수록 softmax함수에서 기울기의 변화가 거의 없는 부분으로 가기 때문입니다.)
- softmax를 거친 값을 value에 곱해준다면, query와 유사한 value일수록, 즉!! 중요한 value일수록 더 높은 값을 가지게 됩니다. 중요한 정보에 더 관심을 둔다는 attention의 원리에 알맞게 된느것 입니다.  


### Multi-Head Attention
![](https://pozalabs.github.io/assets/images/mha.PNG)
- 위의 그림을 수식으로 나타내면 아래와 같습니다.
    - MultiHead(Q,K,V) = Concat(head1,....head_h)W^O 

- ![](https://pozalabs.github.io/assets/images/multi%20head.png)

- d_model dimention의 key,value,query 들로 하나의 attention을 수행하는 대신 key,value,query들에 각각 다른 학습된 linear projection 을 h번 수행하는게 더 좋다고 합니다. 즉, 동일한 Q,K,V에 각각 다른 weight matrix W를 곱해주는것 입니다. 이때 parameter matrix는 아래 그림과 같습니다.
- ![](img/20191021_001427.png)
- 순서대로 query, key, value, out에 대한 parameter matrix 입니다.
     projection이라고 하는 이유는 각각의 값들이 parameter matrix와 곱해졌을때 dk,dv,d_model차원으로 project되기 때문입니다. 논문에선 dk = dv = d_model/h를 사용했는데 꼭 dk와 dv가 같을 필요는 없습니다. 
- 이렇게 project된 key, value, query들은 병렬적으로 attention function을 거쳐 dv dimension output 값으로 나오게 됩니다.
- 그 다음 여러개의 head를 concatenate하고 다시 projection을 수행합니다. 그래서 최종적인 d_model dimention output 값이 나오게 됩니다.
- 각각의 과정에서 dimention을 표현하면 아래와 같습니다. 
![](https://pozalabs.github.io/assets/images/%EC%B0%A8%EC%9B%90.png)

* dQ,dK,dV 는 각각 query, key, value 개수입니다.

#### Self-Attention
###### encoder self-attention layer
![](https://pozalabs.github.io/assets/images/encoder%20self%20attention.png)

- key, value, query들은 모두 encode의 이전 layer의 output에서 옵니다, 따라서 이전 layer의 모든 position에 attention을 줄 수 있습니다. 만약 첫번째 layer 라면 positional encoding이 더해진 input embedding 이 됩니다. 

###### decoder self-attention layer
![](https://pozalabs.github.io/assets/images/decoder%20self%20attention.png)

- encoder 와 비슷하게 decoder에서도 self-attention을 줄 수 있습니다. 하지만 i번째 output을 다시 i+1번째 input으로 사용하는 auto-regressive한 특성을 유지하기 위해, masking out된 scaled dot product attention을 적용합니다.
- masking out이 됐다는 것은 i번째 position에 대한 attention을 얻을 때 i번째 이후에 있는 모든 position은 Attention(Q,K,V) = softmax(QK.T/sqrt(dk))V 에서 softmax의 input 값을 −∞ 로 설정한 것입니다. 이렇게 한다면, i번째 이후에 있는 position에 attention을 주는 경우가 없습니다. 

#### Encoder - Decoder Attention Layer 
![](https://pozalabs.github.io/assets/images/encoder-decoder%20attention.png)

- query들은 이전 decoder layer에서 오고 key와 value들은 encoder의 output에서 오게 됩니다. 그래서 decoder의 모든 position에서 input sequence 즉, encoder output의 모든 position에 attention을 줄 수 있게 됩니다. 
- query가 decoder layer의 output인 이유는 query라는 것이 조건에 해당하기 때문입니다. 좀더 디테일하게 말하자면, '지금 decoder에서 이런 값이 나왔는데 무엇이 output이 돼야 할까" 가 query인것 입니다. 
- 이때 query는 이미 이전 layer에서 masking out 됐으므로, i번째 position 까지만 attention을 얻게 됩니다. 이 같은 과정은 s2s의 전형적인 encoder-decoder emchanisms을 따라한것 입니다. 
- 모든 position에서 attention을 줄 수 있다는게 이해가 안되면   http://mlexplained.com/2017/12/29/attention-is-all-you-need-explained/ 링크를 참조해주세요 

#### Position-wise Feed Foward Networks
- encoder 와 decoder의 각각의 layer 는 아래와 같은 fully connected feed foward network를 포함하고 있습니다. 
![](https://pozalabs.github.io/assets/images/Sample-of-a-feed-forward-neural-network.png)

- position 마다 , 즉 개별 단어마다 적용되기 때문에 position-wise입니다. 
    network는 두 번의 linear transformation과 activation function ReLU로 이루어져 있습니다. 
    
![](https://pozalabs.github.io/assets/images/ffn.png)

- x에 linear transformation을 적용한 뒤, ReLU(max(0,z))를 거쳐 다시 한번 linear transformation을 적용합니다.
- 이때 각각의 position마다 같은 parameter W,b를 사용하지만, layer가 달라지면 다른 parameter를 사용합니다. 
- kernel size가 1이고 channel이 layer 인 convolution을 두 번 수행한 것으로도 위 과정을 이해할 수 있습니다. 

#### Positional Encoding
- transformer 는 recurrence도 아니고 convoulution도 아니기 때문에, 단어의 sequence를 이용하기 위해서느 단어의 position에 대한 정보를 추가해줄 필요가 있습니다.
- 그래서 encoder와 decoder의 input embedding에 postional encoding을 더해 줍니다. 
- positional encoding은 d_model(embedding 차원)과 같은 차원을 갖기 때문에 positional encoding vector 와 embedding vector는 더해질 수 있습니다. 
- 논문에서는 다른 *frequency를 가지는 sine과 cosine 함수를 이용했습니다.
    * 주어진 구간내에서 완료되는 cycle의 개수 
![](img/20191021_011437.png)
- pos 는 position, i는 dimension이고 주기가 10000^(2i/d_model) 2파이 인 삼각함수 입니다. 즉 pos는 sequnece에서 단어의 위치이며, 해당 단어는 i에 0부터 d_model/2 까지를 대입해 d_model차원의 positional encoding vector를 얻게 됩니다. k = 2i+1일 때는 cosine 함수를 k=2i 일 때는 sine 함수를 이용합니다. 이렇게 positional encoding vector를 pos마다 구한다면 비록 같은 column이라고 할지라도 pos가 다르다면 다른 값을 가지게 됩니다. 즉 pos마다 다른 pos와 구분되는 positional encoding값을 얻게 되는 것입니다. 
![](img/20191021_012045.png)
- 이때 PE_(pos+k)는 PE_pos의 linearfunction로 나타낼 수 있습니다 . 표기를 간단히 하기 위해 c = 10000^(2i/d_model)이라 하겠습니다,
        * sin(a+b) = sin(a)cos(b) + cos(a)sin(b)이고
        * cos(a+b) = cos(a)cos(b) - sin(a)sin(b)입니다.
![](img/20191021_012457.png)

- 이러한 성질 때문에 model이 relative postion에 의해 attention하는 것을 더 쉽게 배울 수 있습니다. 
- 논문에서는 학습된 positional embedding 대신 sunusoidal version을 채택했습니다. positonal embedding 의 경우 training 보다 더 긴 sequence가 inference시에 입력으로 들어온다면 문제가 되지만 sinusoidal의 경우 constant 하기 때문에 문제가 되지 않습니다. 단순히 계산량만 늘어날 뿐입니다. 


### Regularization

#### Residual Connection
- Identify Mapping in Deep Residual Networks 라는 논문에서 제시된 방법으로, 아래의 수식이 residual connection을 나타낸 것입니다. 
    y_l = h(x_l) + F(x_l,W_l) 
       x_(l+1) = f(y_l)
- 이때 h(x_l)은 x_l과 같습니다. 논문 제목에서 나온 것처럼 identity mapping 을 해주는것 입니다. 
- 특정한 위치에서의 x_L 을 다음과 같이 x_l과 residual 함수의 합으로 표시할 수 있습니다 .
![](img/20191021_013712.png)

- 그리고 미분을 한다면 다움과 같이 됩니다. 
![](img/20191021_013758.png)

- 이때,σ0/ϵσxL 은 상위 layer 의 gradient 값이 변하지 않고 그대로 하위 layer에 전달되는 것을 보여 줍니다. 즉, layer를 거칠수록  gradient가 사라지는 vanishing gradient 문제를 완화해주는 것 입니다. 
- 또한 forward path나 backward path나 backward path를 간단하게 표현이 가능해 집니다. 

#### Layer Normalization

#### Dropout
- Dropout : a simple way to prevent neural networks from overfitting 이라는 논문에서 제시된 방법입니다. 
    * overfitting??? :
![](https://pozalabs.github.io/assets/images/dropout.PNG)

- dropout 이라는 용어는 neural network 에서 unit들을 dropout 하는 것을 가리킵니다. 즉 해당 unit을 network에서 일시적으로 제거하는 것입니다. 그래서 다른 unit과의 모든 connection이 사라지게 됩니다. 어떤 unit을 dropout할지는 random하게 정해집니다. 

- dropout은 training data에 overfitting되는 문제를 어느정도 막아줍니다. dropout 된 unit들은 training 되지 않는 것이니 training data에 값이 조정되지 않기 떄문입니다.


#### Label Smoothing 
- training 동안 실제 정답인 label의 logit은 다른 logit보다 훨씬 큰값을 갖게 됩니다. 이렇게 해서 model이 주어진 input x 에 대한 label y를 맞추는것입니다.

- 하지만 이렇게 된다면 문제가 발생합니다. overfitting 될 수도 있고 가장 큰 logit을 가지는 것과 나머지 사이의 차이를 점점 크게 만들어 버립니다. 결국 model이 다른 data에 적응하는 능력이 감소됩니다. 

- model이 덜 confident하게 만들기 위해, label distribution q(k|x)=δk,y를 (k가 y일 경우 1, 나머지는 0) 다음과 같이 대체할 수 있습니다.
![11](img/20191021_015703.png)

- 각각 label에 대한 분포u(k), smoothing parameter ϵ입니다. 위와 같다면, k=y일때도 model 은 p(y|x) = 1이 아니라 p(y|x) = (1-ϵ)가 됩니다. 즉 100퍼센트 확신을 하는것이 아니라 조금은 다른 방법을 고려 한다는 것을 뜻합니다.


# Conclusion 
- transformer는 recurrence를 이용하지 않고도 빠르고 정확하게 sequential data를 처리 할 수 있는 model임을 알 수 있다.
- 여러 기법이 사용됐지만, 가장 핵심적인 것은 encoder와 decoder에서 atttention을 통해 query오 가장 밀접한 연관성을 가지는 value를 강조할 수 있고 병렬화가 가능해진 것입니다. 